<a href="https://colab.research.google.com/github/chevy-67/DL-Architecture-Performance-Analysis/blob/main/Case_Study_CV_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("maxsee/v6-6500")

print("Path to dataset files:", path)

100%|██████████| 10.5G/10.5G [02:00<00:00, 93.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout,BatchNormalization
from keras.optimizers import Adam
from keras.utils import image_dataset_from_directory
import os
from PIL import Image

**DATASET PREPROCESSING**

In [ ]:
import os
from PIL import Image

input_dir1 = '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Natural'
output_dir1 = '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Natural_P'
input_dir2 = '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Synthetic'
output_dir2 = '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Synthetic_P'
target_size = (224, 224)

def resize(inp_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    image_files = [f for f in os.listdir(inp_dir) if f.lower().endswith(('.jpg', '.png'))]
    total = len(image_files)

    for i, filename in enumerate(image_files, start=1):
        img_path = os.path.join(inp_dir, filename)
        output_path = os.path.join(out_dir, filename)

        try:
            with Image.open(img_path) as img:
                resized_img = img.resize(target_size)
                resized_img.save(output_path)
                print(f'\rCompleted {i}/{total}', end='', flush=True)
        except Exception as e:
            print(f"\nFailed to process {filename}: {e}")

    print("\nDone!")

resize(input_dir1, output_dir1)
resize(input_dir2, output_dir2)


Completed 45/3275

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Completed 400/3275
Failed to process 113uncy.jpg: cannot write mode RGBA as JPEG
Completed 1106/3275
Failed to process 115sokv.jpg: cannot write mode RGBA as JPEG
Completed 1342/3275
Failed to process 5zs9c8.jpg: cannot identify image file '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Natural/5zs9c8.jpg'
Completed 2665/3275

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (123221280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Completed 3213/3275
Failed to process 112xvt4.jpg: cannot identify image file '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100/Natural/112xvt4.jpg'
Completed 3271/3275
Failed to process 115pufn.jpg: cannot write mode RGBA as JPEG
Completed 3275/3275
Done!
Completed 324/3268
Failed to process zb9gtk.jpg: cannot write mode RGBA as JPEG
Completed 402/3268
Failed to process yv22zs.jpg: cannot write mode RGBA as JPEG
Completed 1861/3268
Failed to process 10henz5.jpg: cannot write mode RGBA as JPEG
Completed 3268/3268
Done!


In [ ]:
import shutil

shutil.rmtree(input_dir1)
shutil.rmtree(input_dir2)
print('Removed old data')

Removed old data


#**Loading the Dataset**

In [ ]:
train_path = '/root/.cache/kagglehub/datasets/maxsee/v6-6500/versions/1/V5Minor100'

In [ ]:
train = image_dataset_from_directory(
    train_path,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    seed=42,
    validation_split=0.2,
    subset="training"
)

validation = image_dataset_from_directory(
    train_path,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    seed=42,
    validation_split=0.2,
    subset="validation"
)

Found 6535 files belonging to 2 classes.
Using 5228 files for training.
Found 6535 files belonging to 2 classes.
Using 1307 files for validation.


##AlexNet Architecture

In [ ]:
alexnet = Sequential([
    Conv2D(96, kernel_size=(11, 11), strides=5, activation='relu', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(3, 3), strides=2),

    Conv2D(256, kernel_size=(11, 11), strides=1,padding='same',activation='relu'),
    MaxPool2D(pool_size=(3, 3), strides=2),
    BatchNormalization(),

    Conv2D(384, kernel_size=(3, 3), strides=1,padding='same' ,activation='relu'),
    BatchNormalization(),

    Conv2D(384, kernel_size=(3, 3), strides=1,padding='same' ,activation='relu'),
    BatchNormalization(),

    Conv2D(256, kernel_size=(3, 3), strides=1,padding='same' ,activation='relu'),
    MaxPool2D(pool_size=(3, 3), strides=2),
    BatchNormalization(),

    Flatten(),
    Dropout(0.3),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(4096, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
alexnet.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
alexnet.fit(train,epochs=5)

Epoch 1/5
 10/164 ━━━━━━━━━━━━━━━━━━━━ 13:03 5s/step - accuracy: 0.5526 - loss: 5.1676

##EfficientNet Model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalAvgPool2D

In [ ]:
input_shape = (224,224,3)
effnet_base_model = EfficientNetB0(weights='imagenet',include_top=False,input_shape=input_shape)
effnet_base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
inputs = Input(shape=input_shape)
x = effnet_base_model(inputs, training=False)
x = GlobalAvgPool2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

efficient_model = Model(inputs, outputs)

In [ ]:
efficient_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

##Xception Model

In [ ]:
from tensorflow.keras.applications import Xception

In [ ]:
xception_base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
xception_base_model.trainable = False

inputs = Input(shape=input_shape)
x = xception_base_model(inputs, training=False)
x = GlobalAvgPool2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

xception_model = Model(inputs, outputs)


In [ ]:
xception_base_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

##MobileNet Model

In [ ]:
from tensorflow.keras.applications import MobileNet

In [ ]:
mobilenet_base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
mobilenet_base_model.trainable = False

inputs = Input(shape=input_shape)
x = mobilenet_base_model(inputs, training=False)
x = GlobalAvgPool2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

mobilenet_base_model = Model(inputs, outputs)


In [ ]:
mobilenet_base_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])